# Exemplo: Pressões de vapor e entalpias de vaporização de alquil formamidas

## Introdução

Este exemplo aborda o avaliação da pressão de vapor em função da temperatura da N-benzil-formamida,  substância na figura 1, pura.


**Figura 1: N-benzil-formamida. Fonte: [1]**

<img src="formamida.jpg" alt="drawing" width="150"/>



Os dados experimentais são apresentados pela Tabela 1.

**Tabela 1. Dados experimentais. Fonte: [2].**

Temperatura / K | Pressão de vapor (Pa) | Incertezas para pressão de vapor
:--------------:|:---------------------:|:-------------------------------:
        297.1   |         2.93          |        0.08 
        298.2   |         3.21          |        0.09
        299.3   |         3.49          |        0.09
        301.2   |         4.22          |        0.11
        304.2   |         5.60          |        0.17
        307.2   |         7.31          |        0.21
          .     |           .           |          .
          .     |           .           |          .
          .     |           .           |          .
        334.2   |          63.36        |        1.61
        337.1   |          78.93        |        2.00
        340.2   |          93.65        |        2.37
        343.2   |          115.11       |        2.90
        346.2   |          140.27       |        3.53
        349.1   |          171.89       |        4.32
        352.2   |          208.00       |        5.23
        
        
O modelo para esta reação é dado por [1]:

$y = \exp\left(A-(\frac{B}{T+C}\right))$ (1),

em que $y$ corresponde a pressão de vapor da substância, $T$ é a temperatura, e por fim $A$, $B$ e $C$ são os parâmetros a serem estimados.

Então, o problema de otimização a ser resolvido envolve o uso da função objetivo de mínimos quadrados ponderados pelo inverso da variância, conforme [2]:

$\min_{A, B, C} \sum_{i=1}^{NE} \left(\frac{y^{exp}_i-y_i(A,B,C)}{u^2_{y_i}}\right)$ (2),

sujeito à (1).

Para resolução deste problema no MT_PEU serão usados os símbolos:

* Símbolos das grandezas independentes (tempo e temperatura): T
* Símbolos das grandezas dependentes (pressão de vapor): y
* Símbolos dos parâmetros: A, B e C

## Importação de pacotes

importação de bibliotecas (pacotes) necessárias para execução do código.

* **MT_PEU**: biblioteca que contém as principais funcionalidades da ferramenta

    * Importar a classe **EstimacaoNaoLinear**, que será utilizada neste exemplo de estimação não linear
    
* **numpy**: biblioteca para computação científica -> será necessária somente a função exp (exponencial)

    * será necessária somente a função **exp** (exponencial) para construção do modelo

In [1]:
from MT_PEU import EstimacaoNaoLinear
from numpy import exp

## Criação de modelo

O modelo (1) representa o comportamento da grandeza dependente, no qual os parâmetros $A$, $B$ e $C$ serão estimados.

Este modelo é definido, então, na forma de uma subrotina, **def**, do python e representado por:

In [2]:
def Modelo(param,x,args):


    T = x[:,0]
    A, B, C = param[0], param[1],  param[2]
    
    
    return exp(A-(B/(T+C))) # cálculo de Pvp - vetorizado

## Inicialização de classe

Para executar a estimação, o primeiro passo é configurar a classe **EstimacaoNaoLinear**, inicialmente, incluindo informações básicas:

* O modelo,
* Os símbolos para as grandeza independentes *(x)*; 
* Os símbolos para as grandezas dependentes *(y)*; 
* Os símbolos para os parâmetros *(param)*;
* O nome do projeto, o nome da pasta onde serão gerados os resultados

In [3]:
Estimation = EstimacaoNaoLinear(Modelo, simbolos_x=[r'T'], simbolos_y=[r'y'], 
                            simbolos_param=['A','B','C'],  Folder='Exemplo5' )

## Inclusão de dados

Os dados experimentais disponibilizados em [2] da variável dependente (y) e da variáveis independentes (T), são apresentados abaixo na forma de listas:

In [4]:
#Pressão de vapor
y = [2.93,3.21,3.49,4.22,5.60,7.31,9.12,13.07,14.98,17.63,18.02,22.08,26.95,34.61,
       40.93,50.17,63.36,78.93,93.65,115.11,140.27,171.89,208.00]

#Temperatura
temperatura = [297.1,298.2,299.3,301.2,304.2,307.2,310.2,314.1,316.2,317.8,318.2,320.2,
               323.1,326.2,329.1,331.2,334.2,337.1,340.2,343.2,346.2,349.1,352.2]

Como entrada **obrigatória**, a plataforma MT_PEU necessita da **incerteza dos dados experimentais** (ux1, ux2, uy1).
Neste exemplo, foram adotados os valores das incertezas presentes em [2].

In [5]:
uxtemperatura = [0.1]*23
uy = [0.08,0.09,0.09,0.11,0.17,0.21,0.25,0.35,0.40,0.47,0.48,0.58,
      0.70,0.89,1.05,1.28,1.61,2.00,2.37,2.90,3.53,4.32,5.23]

 **Inclusão de dados experimentais na estimação:**

O método *setDados* é utilizado para incluir os dados para as grandezas dependentes e independentes. Sintaxe:

* identificação da grandeza, se independente ou dependente: 0 ou 1 (respectivamente)
* os dados experimentais e suas incertezas inseridos em sequência na forma de tuplas.

In [6]:
Estimation.setDados(0,(temperatura,uxtemperatura))
Estimation.setDados(1,(y,uy))

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

In [7]:
Estimation.setConjunto(tipo='estimacao')

AttributeError: 'EstimacaoNaoLinear' object has no attribute '_EstimacaoLinear__coluna_dumb'

## Otimização

Neste exemplo, o usuário tem a opção de escolha do algoritmo a ser utilizado na otimização. 
Disponiveis: 'ipopt', 'bonmin'. 
Caso opte por não escolher, será utilizado o algoritmo default: ipopt.

Para realizar a otimização é obrigatório informar a estimativa inicial que, neste caso, será [1, 1.5, 0.009].

In [ ]:
Estimation.optimize(initial_estimative = [1, 1.5, 0.009])

## Incerteza


Para avaliação da incerteza dos parâmetros, será utilizado o método 'Geral', outras opções são: 2InvHessiana, SensibilidadeModelo. 

Para melhor apresentar a região de abrangência pelo método da verossimilhança, pode-se mapear a função objetivo, através do argumento objectiveFunctionMapping = True.

Caso o usuário **não** tenha interesse em **avaliar a incerteza dos parâmetros**, apenas **não** executar **Estime.incertezaParametros**.

In [ ]:
Estimation.incertezaParametros(metodoIncerteza='Geral', objectiveFunctionMapping=True)

## Predição e análise de resíduos

 No método predição, é feita a avaliação da grandeza dependente com base nos parâmetros avaliados, no modelo e nos dados das grandezas dependentes. Caso tenham dados para predição (no método setConjunto foi definido o tipo "predicao"), estes são usados. Além disso, este método avalia a matriz de covariância da predição, caso o método incertezaParametros tenha sido executado.
 
 Em análise de resíduos, é possível verificar alguns indicares sobre os resíduos da estimação, entre eles: (i) avaliação da média do resíduo, (ii) homocedasticidade dos resíduos (possíveis relações de dependencia e/ou tendencia entre as variaveis), (iii)  normalidade, (iv) $R^2$, (v) autocorrelação. Além de avaliação sobre o valor ótimo da função objetivo.
 
 A analise de residuos é feita prioritariamente com os dados de validação.

In [ ]:
Estimation.predicao()
Estimation.analiseResiduos()

## Gráficos e relatório

Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito.
dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.

In [ ]:
etapas = ['otimizacao', 'grandezas-entrada', 'predicao', 'grandezas-calculadas', 'analiseResiduos', 'regiaoAbrangencia']
Estimation.graficos(etapas)
Estimation.relatorio()

Referências: 

[1] SANJARI, Ehsan. A new simple method for accurate calculation of saturated vapor pressure. Elsevier. 
p. 12-16. mar. 2013.

[2] ZAITSEVA, Ksenia V.; ZAITSAU, Dzmitry H.; VARFOLOMEEV, Mikhail A.. 
Vapour pressures and enthalpies of vaporisation of alkyl formamides. Elsevier. Alemanha, p. 228-238. maio 2019.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.